# Generating graphs of E vs. t, B vs. t, and V vs. t

# M = 10^6, D* = 10^(-6), P0 = 0

## Program Setup

In [11]:
# Import programs

import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
import math

In [12]:
# Initialize constants for plots
num_bins = 200

axis_font = {'size':'40'}
title_font = {'size':'40'}

In [13]:
# Physical parameters
meandeltaT = np.pi
J = 300
dTau = 2 * np.pi / J
M = 10 ** 6
K = 1
N0 = 1
P0 = 0
Dstar = 10 ** (-6)
eps = 0.1
std = 0.01
eqmean = 0
Tr = K * meandeltaT

In [14]:
# Initial conditions
initial_B = 0
initial_u = 0

## Functions

In [15]:
# function that generates an array of values of DeltaT, time b/t collisions
def fundeltaT(num_bins, M, dTau):
    deltaT = []
    
    rayleigh = np.random.rayleigh(meandeltaT, M)
    hist, bin_edges = np.histogram(rayleigh, bins = num_bins, density = True)
    cum_values = np.zeros(bin_edges.shape)
    cum_values[1:] = np.cumsum(hist*np.diff(bin_edges))
    inv_cdf = interpolate.interp1d(cum_values, bin_edges)
    
    for i in range(M):
        r = np.random.rand(1)
        deltaTtemp = inv_cdf(r)

        if (deltaTtemp % dTau < 0.005):
            deltaTtemp2 = deltaTtemp - (deltaTtemp % dTau)
        else:
            deltaTtemp2 = deltaTtemp + dTau - (deltaTtemp % dTau)
        deltaT.append(deltaTtemp2)
    
    return deltaT

In [16]:
# Generating an array of values of V
def funV(std, deltaT, eqmean, Tr):
    V = []
    y1 = 0
    
    for i in range(len(deltaT) - 1):
        Tn = (deltaT[i] + deltaT[i + 1]/2)
        
        stdevt = ((1 - math.exp(-2 * Tn/Tr)) * std ** 2) ** (1/2)
        meant = eqmean + math.exp(-Tn / Tr) * (y1 - eqmean)
        
        y1 = np.random.normal(meant, stdevt, 2)[0]
        V.append(y1)
        
    return V

In [17]:
# generating array of values of B and E
def funB_E(Dstar, dTau, B, N0, P0, deltaT, M, eps, V, u):
    # created using Heun's method of integration 
        
    Amp = [B]
    t = 0.0
    amptime = [t]
    E = []
    
    for i in range (0, M - 1):
        
        for j in range(0, len(deltaT) - 1):
            m1 = u
            k1 = - (Dstar * u) - B  - (N0 * B**2) - (P0 * B**3)
            m2 = u + dTau * k1
            B_2 = B + dTau * m1
            u_2 = m2
            k2 = - Dstar * u_2 - B_2 - (N0 * B_2**2) - (P0 * B_2**3)
            m2 = u + dTau * k2
            t = t + dTau
            B += (dTau / 2) * (m1 + m2)
            u += (dTau / 2) * (k1 + k2)

            # include? needed in outer loop?
            if math.isnan(B) or abs(B) > 1:
                break

            Amp.append(B)
            amptime.append(t)
            E.append((1/2 * u**2) + (1/2 * B**2) + (1/3 * N0 * B**3) + (1/4 * P0 * B**4))
        
        V_i = V[i]
        dbdtf = u
        u = dbdtf + 2 * eps * (V_i - dbdtf)
    
    return Amp, amptime, E

In [18]:
def funplots(amptime, E, Amp, V):
    fig, axs = plt.subplots(3, 1, figsize=(40,15))
    fig.subplots_adjust(hspace=0) 
    
    axs[2].plot(amptime[:99], V, '.k')
    axs[2].set_ylabel('V', **axis_font)
    axs[2].set_xlabel('T', **axis_font)
    axs[2].tick_params(axis = 'both', labelsize = 20)
    
    axs[1].plot(amptime, Amp, 'k')
    axs[1].set_ylabel('B', **axis_font)
    axs[1].set_xlabel('T', **axis_font)
    axs[1].tick_params(axis = 'both', labelsize = 20)
    
    axs[0].plot(amptime[:-1], E, 'k')
    axs[0].set_ylabel('E', **axis_font)
    axs[0].set_xlabel('T', **axis_font)
    axs[0].tick_params(axis = 'both', labelsize = 20)
    
    plt.show()


## Generating Graphs

In [19]:
# running functions for sample plot  
deltaT = fundeltaT(num_bins, M, dTau)
V = funV(std, deltaT, eqmean, Tr)
Amp, amptime, E = funB_E(Dstar, dTau, initial_B, N0, P0, deltaT, M, eps, V, initial_u)

KeyboardInterrupt: 

In [ ]:
funplots(amptime, E, Amp, V)